In [ ]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.datasets import mnist
from sklearn.model_selection import train_test_split

In [ ]:
class DataLoader:
  @staticmethod
  def load_data():
    (x_train, y_train), (x_test, y_test) = mnist.load_data()
    x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, test_size=0.2, random_state = 42)
    return x_train, y_train, x_val, y_val, x_test, y_test

In [ ]:
class DataPreprocessor:
  @staticmethod
  def preprocess_data(x_train, x_val, x_test):
    x_train = x_train.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_val = x_val.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    x_test = x_test.reshape((-1, 28, 28, 1)).astype('float32') / 255.0
    return x_train, x_val, x_test

In [ ]:
class ModelBuilder:
  @staticmethod
  def build_model(input_shape, num_classes):
    model = models.Sequential()

    # Block 1
    model.add(layers.Conv2D(64, (3, 3), activation='relu', input_shape=input_shape, padding='same'))
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2,2), strides=(2, 2)))

    # Block 2
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Block 3
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu', padding='same'))
    model.add(layers.Conv2D(256, (3, 3), activation = 'relu', padding='same'))
    model.add(layers.MaxPooling2D((2, 2), strides=(2, 2)))

    # Fully Connected Layers
    model.add(layers.Flatten())
    model.add(layers.Dense(4096, activation = 'relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(4096, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(num_classes, activation='softmax'))

    return model

In [ ]:
class Trainer:
    def __init__(self, model, optimizer, loss_function, metrics):
        self.model = model
        self.model.compile(optimizer=optimizer, loss=loss_function, metrics=metrics)

    def train_model(self, x_train, y_train, x_val, y_val, epochs, batch_size, validation_data):
        history = self.model.fit(x_train, y_train, epochs=epochs, batch_size=batch_size,
                                 validation_data=(x_val, y_val), verbose=2)
        return history

In [ ]:
def main():
    x_train, y_train, x_val, y_val, x_test, y_test = DataLoader.load_data()
    x_train, x_val, x_test = DataPreprocessor.preprocess_data(x_train, x_val, x_test)

    input_shape = x_train.shape[1:]
    num_classes = 10

    model = ModelBuilder.build_model(input_shape, num_classes)

    optimizer = 'adam'
    loss_function = 'sparse_categorical_crossentropy'
    metrics = ['accuracy']

    trainer = Trainer(model, optimizer, loss_function, metrics)

    epochs = 5
    batch_size = 32

    history = trainer.train_model(x_train, y_train, x_val, y_val, epochs, batch_size, validation_data=(x_val, y_val))

    test_loss, test_accuracy = model.evaluate(x_test, y_test, verbose=2)
    print(f'Test Accuracy: {test_accuracy * 100:.2f}%')

In [ ]:
if __name__ == "__main__":
    # Check if GPU is available and being used
    physical_devices = tf.config.list_physical_devices('GPU')
    if len(physical_devices) == 0:
        print("No GPU devices available. Training on CPU.")
    else:
        tf.config.experimental.set_memory_growth(physical_devices[0], True)
        print(f"GPU {physical_devices[0]} is available. Training on GPU.")

    main()

GPU PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU') is available. Training on GPU.
11490434/11490434 [==============================] - 1s 0us/step
Epoch 1/5
1500/1500 - 39s - loss: 0.4424 - accuracy: 0.8433 - val_loss: 0.0663 - val_accuracy: 0.9812 - 39s/epoch - 26ms/step
Epoch 2/5
1500/1500 - 25s - loss: 0.0782 - accuracy: 0.9786 - val_loss: 0.0477 - val_accuracy: 0.9857 - 25s/epoch - 17ms/step
Epoch 3/5
1500/1500 - 24s - loss: 0.0643 - accuracy: 0.9831 - val_loss: 0.0588 - val_accuracy: 0.9843 - 24s/epoch - 16ms/step
Epoch 4/5
1500/1500 - 24s - loss: 0.0621 - accuracy: 0.9844 - val_loss: 0.0492 - val_accuracy: 0.9869 - 24s/epoch - 16ms/step
Epoch 5/5
1500/1500 - 23s - loss: 0.0561 - accuracy: 0.9857 - val_loss: 0.0411 - val_accuracy: 0.9895 - 23s/epoch - 15ms/step
313/313 - 2s - loss: 0.0426 - accuracy: 0.9884 - 2s/epoch - 5ms/step
Test Accuracy: 98.84%
